In [52]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from one.api import ONE
from brainbox.io.one import SessionLoader

DSETS = ['photometry.signal.pqt', 'photometryROI.locations.pqt']

In [4]:
one = ONE()

In [115]:
eids = list(one.search(subject='ZFM-08757', datasets=DSETS))
len(eids)

4

In [116]:
eid = '0d118e83-1450-4382-9125-5fabc5c31b88'
eid = eids[-1]

In [117]:
signals = one.load_dataset(id=eid, dataset='photometry.signal.pqt')
locations = one.load_dataset(id=eid, dataset='photometryROI.locations.pqt').reset_index()
rois = locations['ROI'].to_list()
photometry = signals[list(rois) + ['name']].set_index(signals['times']).dropna()
print("ROIs: ", rois)

ROIs:  ['G0']


In [118]:
buffer = 15
loader = SessionLoader(one, eid=eid)
## FIXME: appropriately handle cases with multiple task collections
loader.load_trials(collection='alf/task_00')
timings = [col for col in loader.trials.columns if col.endswith('_times')]
t0 = loader.trials[timings].min().min()
t1 = loader.trials[timings].max().max()
i0 = photometry.index.searchsorted(t0 - buffer)
i1 = photometry.index.searchsorted(t1 + buffer)
photometry = photometry.iloc[i0:i1].copy()

In [119]:
gcamp = photometry.query('name == "GCaMP"')
iso = photometry.query('name == "Isosbestic"')

In [120]:
%matplotlib tk

fig, ax = plt.subplots(figsize=(16, 4))
ax.plot(gcamp[rois[0]])
ax.plot(iso[rois[0]], color='gray')